# Current clamp: Part 2
In part 2 if the current clamp section we will extend the anaysis of spike features such as how spike shape changes with spike number. We will cover how rheobase, membrane resistance and spike width are related. We also cover differences between cells types.

In [ ]:
from collections import defaultdict
from pathlib import Path

import numpy as np
import pandas as pd
from bokeh.io import output_notebook, show
from bokeh.layouts import column, gridplot, row
from bokeh.models import (
    Checkbox,
    ColumnDataSource,
    CustomJS,
    Select,
    Slider,
    Span,
    Spinner,
)
from bokeh.plotting import figure

output_notebook()

In [ ]:
url_path = "https://cdn.jsdelivr.net/gh/LarsHenrikNelson/PatchClampHandbook/data/"
pyramidal = pd.read_csv(url_path + "pyramidal/spike_data.csv")
msn = pd.read_csv(url_path + "msn/spike_data.csv")
pv = pd.read_csv(url_path + "pv/spike_data.csv")
pyramidal = pyramidal.rename(columns={"Peak (ms)": "Spike (ms)", "Peak (mV)": "Spike (mV)"})

In [ ]:
print(msn.columns)
print(pyramidal.columns)
print(pv.columns)

## Changes in spike shape with increasing number of spikes
One important feature of spikes is how their shape adapts to increasing firing rate. Since there is a limit to how fast and often some ion channels can open the spike shape will change with increasing spike frequency. One interesting thing to look as is how the area under the curve (AUC) of the spike changes. You can see how the depolarization phase of the spike remains virtually unchanged where as the repolarization phase gets much longer. The reason the repolorization phase gets so much larger is due to progressive inactivation of BK channels (Ca2+ activated K+ channel) or Kv4 channels (voltage-gated K+ channel). By splitting the AUC into pre peak and post peak components we can see that Na+ currents likely remain relatively unchanged however, depolarization of the cell is prolonged due decreased K+/Ca2+ currents. Some cell types, such as parvalbumin interneurons, have extremely stable spike waveform features. Lastly, notice how some relationships look to be mostly linear whereas some seem to be linear. Spike threshold, Spike (mV), Depolarization time (ms) and Pre-peak AUC are all pretty much linear. This likely because there is a single type of channel regulating these features (i.e Na+ channels). The features that are regulated by K+ and Ca2+ channels seem to be logarithmic or, some other curve in the case of the AHP.

In [ ]:
columns = [
    "Threshold (mV)",
    "HW (ms)",
    "HW (mV)",
    "FW (ms)",
    "AUC",
    "AUC Left",
    "AUC Right",
    "Min Velocity (ms)",
    "Min Velocity",
    "Max Velocity (ms)",
    "Max Velocity",
    "AHP (ms)",
    "AHP (mV)",
    "Spike (ms)",
    "Spike (mV)",
    "Spike Number",
]

In [ ]:
def create_multline_source(data):
    data_groupby = data.groupby("Acq Number")
    data_groupby.groups
    data_source = defaultdict(list)
    for c in columns:
        for key, value in data_groupby.groups.items():
            data_source[c].append(data.loc[value, c])
    data_source["y"] = data_source[columns[0]]
    data_source = ColumnDataSource(data_source)
    return data_source

msn_source = create_multline_source(msn)
fig1 = figure(height=200, width=250, title="MSN")
mline = fig1.multi_line("Spike Number", "y", source=msn_source, color="black")

pyramidal_source = create_multline_source(pyramidal)
fig2 = figure(height=200, width=250, title="Pyramidal")
mline = fig2.multi_line("Spike Number", "y", source=pyramidal_source, color="black")
menu = Select(title="Column", value=columns[0], options=columns)

pv_source = create_multline_source(pv)
fig3 = figure(height=200, width=250, title="PV")
mline = fig3.multi_line("Spike Number", "y", source=pv_source, color="black")
menu = Select(title="Column", value=columns[0], options=columns[:-1])

callback = CustomJS(
    args=dict(
        msn_source=msn_source,
        pyramidal_source=pyramidal_source,
        pv_source=pv_source,
        menu=menu,
    ),
    code="""
    msn_source.data.y = msn_source.data[menu.value];
    msn_source.change.emit();
    pyramidal_source.data.y = pyramidal_source.data[menu.value];
    pyramidal_source.change.emit();

    pv_source.data.y = pv_source.data[menu.value];
    pv_source.change.emit();
""",)
menu.js_on_change("value", callback)
show(column(menu, row(fig1, fig2, fig3)))

How would you compare whether there are differences between cells or genotypes or treatments? One of the easiest ways is to curve fit similar to what we did with FI curve and the sigmoid curve. Looking at the data there are couple curves you could choose. The primary curve I would is a logarithmic curve. Some curves look more linear or even polynomial in the case of the AHP. Choosing a curve to fit may also depend on the cell type.

### Changes in IEI with frequency
The time between each spike, interevent interval (IEI) or interspike interval (ISI), can also change with spike frequency. Parvalbumin interneurons tend to have a very stable IEI. They can also burst fire a lower current injects with short IEIs followed by a longer IEI then short IEIs. Pyramidal cells on the other can have a lengthing of the IEI the more spikes there are. However, I have noticed in younger mice (P16) that this only occurs when the cell is spiking at higher frequencies.

In [ ]:
def create_iei_source(data):
    data_groupby = data.groupby("Acq Number")
    data_groupby.groups
    data_source = defaultdict(list)
    for key, value in data_groupby.groups.items():
        data_source["IEI"].append(np.diff(data.loc[value, "Peak (ms)"]))
        data_source["IEI Number"].append(data.loc[value, "Spike Number"][:-1])
    data_source = ColumnDataSource(data_source)
    return data_source

msn_source = create_iei_source(msn)
fig1 = figure(height=200, width=250)
mline = fig1.multi_line("IEI Number", "IEI", source=msn_source, color="black")

pyramidal_source = create_iei_source(pyramidal)
fig2 = figure(height=200, width=250)
mline = fig2.multi_line("IEI Number", "IEI", source=pyramidal_source, color="black")
menu = Select(title="Column", value=columns[0], options=columns)

pv_source = create_iei_source(pv)
fig3 = figure(height=200, width=250)
mline = fig3.multi_line("IEI Number", "IEI", source=pv_source, color="black")
menu = Select(title="Column", value=columns[0], options=columns[:-1])
show(row(fig1, fig2, fig3))

## Relationship between spike features
One important thing to understand is how all these spike features interact. Increases in spike width can offset decreases in spike amplitude thus maintaining the Na+ currents and the synaptic output of the cell. One interesting thing to note is that particularly for the repolarization time you can see how the first spikes of each acquisition are clustered together whereas the later spikes have an almost perfect correlation between peak amplitude and repolarization time. As peak voltage decreases repolarization time increases.

In [ ]:
plots = list(output.data.keys())[:-1]
temp = {i: np.concatenate(output.data[i]) for i in plots}
color = np.concatenate(output.data["x"])
color = np.where(color == 1, "orange", "grey")
temp["color"] = color
temp["x"] = temp[plots[0]]
temp["y"] = temp[plots[1]]
source = ColumnDataSource(temp)
xy_figure = figure(height=250, width=400)
menu1 = Select(
    title="y",
    value=plots[0],
    options=plots,
)
menu2 = Select(
    title="x",
    value=plots[1],
    options=plots,
)

xy_line = xy_figure.scatter(x="x", y="y", color="color", source=source, alpha=0.6)
callback = CustomJS(
    args=dict(
        source=source,
        menu1=menu1,
        menu2=menu2,
    ),
    code="""
    source.data.y = source.data[menu1.value];
    source.data.x = source.data[menu2.value];
    source.change.emit();
""",
)

menu1.js_on_change("value", callback)
menu2.js_on_change("value", callback)

show(column(row(menu1, menu2), xy_figure))


## Relationship between cell features

In [ ]:
url = "https://cdn.jsdelivr.net/gh/LarsHenrikNelson/PatchClampHandbook/data/current_clamp.csv"
df = pd.read_csv(url)

source = ColumnDataSource(df)
xy_figure = figure(height=250, width=400)
menu1 = Select(
    title="y",
    value="Rheobase (pA)",
    options=df.columns.to_list(),
)
xcheck = Checkbox(label="Log(x)")
menu2 = Select(
    title="x",
    value="Membrane resistance",
    options=df.columns.to_list(),
)
ycheck = Checkbox(label="Log(y)")
xy_line = xy_figure.scatter(
    x=source.data["Rheobase (pA)"], y=source.data["Membrane resistance"], color="grey"
)
callback = CustomJS(
    args=dict(
        source=source,
        xy_line=xy_line,
        menu1=menu1,
        menu2=menu2,
        xcheck=xcheck,
        ycheck=ycheck,
    ),
    code="""
    if (ycheck.active) {
        var y = source.data[menu1.value].map(num => Math.log10(num));
    } else {
        var y = source.data[menu1.value]
    }
    if (xcheck.active) {
        var x = source.data[menu2.value].map(num => Math.log10(num));
    } else {
        var x = source.data[menu2.value];
    }
    xy_line.data_source.data.y = y;
    xy_line.data_source.data.x = x;
    xy_line.data_source.change.emit();
""",
)

menu1.js_on_change("value", callback)
menu2.js_on_change("value", callback)
ycheck.js_on_change("active", callback)
xcheck.js_on_change("active", callback)

show(column(row(column(menu1, ycheck), column(menu2, xcheck)), xy_figure))